In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 2
setup_one_gpu(GPU)

Picking GPU 2


In [2]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from general_tools.notebook.tf import reset_tf_graph
from general_tools.in_out.basics import create_dir, files_in_subdirs

from geo_tool import Point_Cloud

from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
from tf_lab.point_clouds.in_out import PointCloudDataSet, load_point_clouds_from_filenames
from tf_lab.point_clouds.encoders_decoders import encoder_with_convs_and_symmetry, decoder_with_fc_only,\
encoder_with_convs_and_symmetry_and_fc

from tf_lab.in_out.basics import read_saved_epochs
from tf_lab.data_sets.articulated_human_poses import pc_loader
from tf_lab.point_clouds.convenience import reconstruct_pclouds
from tf_lab.data_sets.shape_net import snc_category_to_synth_id, load_all_point_clouds_under_folder

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
n_pc_points = 2048
do_training = True
first_time_running = True
seed = 42
class_name = 'chair'
ae_loss = 'emd'
top_data_dir = '/orions4-zfs/projects/optas/DATA/'
syn_id = snc_category_to_synth_id()[class_name]

In [5]:
pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Core/from_manifold_meshes/centered/', str(n_pc_points))
pclouds_path = osp.join(pclouds_path, syn_id)
pcloud_data = load_all_point_clouds_under_folder(pclouds_path)

In [9]:
print ae_conf

                    batch_size: 50
                 consistent_io: None
                         debug: False
                       decoder: decoder_with_fc_only
                  decoder_args: {'layer_sizes': [1024, 2048, 6144]}
                       encoder: encoder_with_convs_and_symmetry
                  encoder_args: {'filter_sizes': [40, 20, 10, 10], 'n_filters': [128, 128, 256, 64], 'strides': [1, 2, 2, 1]}
               experiment_name: ae_farm_chair_conv_arch_11_2048pts_emd
                 gauss_augment: None
                  is_denoising: False
               latent_vs_recon: 1.0
                 learning_rate: 0.0005
                          loss: emd
             loss_display_step: 1
                       n_input: [2048, 3]
                      n_output: [2048, 3]
                           n_z: None
             saver_max_to_keep: None
                    saver_step: 10
                     train_dir: /orions4-zfs/projects/optas/DATA/OUT/icml/nn_models/ae_farming/

In [6]:
# Load AE model
ae_id = '11' # 64 bottleneck for emd
ae_net_name = 'ae_farm_' + class_name + '_conv_arch_' + ae_id + '_' + str(n_pc_points)\
+ 'pts_' + ae_loss

ae_net_dir = osp.join(top_data_dir, 'OUT/icml/nn_models/ae_farming', ae_net_name)
ae_conf = Conf.load(osp.join(ae_net_dir, 'configuration'))

if ae_conf.train_dir != ae_net_dir:
    ae_conf.train_dir = ae_net_dir
    ae_conf.save(osp.join(ae_conf.train_dir, 'configuration'))
    
reset_tf_graph()

ae = PointNetAutoEncoder(ae_net_name, ae_conf)
saved_epochs = read_saved_epochs(ae_conf.train_dir)
# load_epoch = saved_epochs[-1]
# ae.restore_model(ae_conf.train_dir, load_epoch, verbose=True)
# print 'Bottleneck = ', ae.bottleneck_size
# recon_pclouds, loss_val = reconstruct_pclouds(ae, pcloud_data.point_clouds, 100)
# print 'Loss of reconstructed data ', loss_val

Model restored in epoch 1000.
Bottleneck =  64
Loss of reconstructed data  20730.5284926


In [25]:
_, latent_codes, temp_names = ae.embedding_at_tensor(pcloud_data, ae_conf, tensor_name='bottleneck')
assert(np.alltrue(temp_names==pcloud_data.labels))
model_ids = np.array([i.split('_')[1] for i in pcloud_data.labels], dtype=object)